In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
import nltk
import tensorflow as tf
import os
import string
import re
data = pd.read_excel('sentence_past_present_future.xlsx')
y = data['Categorie']
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)
from nltk.tokenize import word_tokenize
from tensorflow.keras.preprocessing.text import one_hot
voc_size = 10000
input = []
for x in data['Sentences']:
    val = word_tokenize(x)
    one_rpr = one_hot(str(val),n=voc_size,lower=False,split=None)
    input.append(one_rpr)
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
sent_length = 15
embedded_docs = pad_sequences(input,padding='post',maxlen=sent_length)
dim = 10
model =Sequential()
model.add(Embedding(voc_size,dim,input_length=sent_length))
model.compile('adam','mse')
model.summary()
x = model.predict(embedded_docs)
rows = x.shape[0] 
cols = x.shape[1] * x.shape[2]
x = x.reshape(rows,cols)
from sklearn.model_selection import train_test_split
x_train , x_test , y_train , y_test = train_test_split(x,y,test_size=0.25,random_state=42)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 15, 10)            100000    
                                                                 
Total params: 100000 (390.62 KB)
Trainable params: 100000 (390.62 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
87/87 [==============================] - 0s 1ms/step


In [2]:
from sklearn.ensemble import RandomForestClassifier

In [3]:
model = RandomForestClassifier(n_estimators=100)

In [4]:
model.fit(x_train,y_train)

RandomForestClassifier()

In [5]:
from sklearn.metrics import accuracy_score , confusion_matrix , precision_score , recall_score , f1_score , roc_auc_score
def Evaluation_metrics(actual,pred):
    accuracy_sco = accuracy_score(actual,pred)
    confusion_mat = confusion_matrix(actual,pred)
    precision_sco = precision_score(actual,pred,average='macro')
    recall_sco = recall_score(actual,pred,average='macro')
    f1_sco = f1_score(actual,pred,average='macro')
    return accuracy_sco,confusion_mat,precision_sco,recall_sco,f1_sco

In [10]:
predicted_val = model.predict(x_test)

In [12]:
Evaluation_metrics(y_test,predicted_val)

(0.84992784992785,
 array([[215,  15,   2],
        [  5, 180,  46],
        [  6,  30, 194]], dtype=int64),
 0.8509934420634341,
 0.8498077260071263,
 0.8501240704455241)

In [13]:
model.score(x_test,y_test)

0.84992784992785

In [14]:
model.score(x_train,y_train)

1.0